In [1]:
import pandas as pd, numpy as np
import itertools, requests, json

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [5]:
dc=dc.set_index('uid')[['relevant','severity','deaths','duplicate']]
dm=dm.set_index('uid')
lm=lm.set_index('uid')[['predicted_locs','actual_locs']]

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.join(dc).drop_duplicates()

In [8]:
dl=df.join(lm)

In [9]:
len(dl)

13

Working df ready. Now infer location.

In [10]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [11]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [12]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [13]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [14]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,"Videóra vették, ahogy elhurcolja a megölt tehe...",0,2019-09-11,2019. szeptember 11.,https://szekelyhon.ro/aktualis/videora-vettek-...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['medve', 'Udvarhelyszék']","Videóra vették, ahogy elhurcolja a megölt tehe...",...,1,3,0,0,"['Pálpatak', 'Pálpataka']","['Pálpatak', 'Pálpataka']",Pálpatak,46.8038,24.8855,"Pálpatak, 0"
1,"Videóra vették, ahogy elhurcolja a megölt tehe...",0,2019-09-11,2019. szeptember 11.,https://szekelyhon.ro/aktualis/videora-vettek-...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['medve', 'Udvarhelyszék']","Videóra vették, ahogy elhurcolja a megölt tehe...",...,1,3,0,0,"['Pálpatak', 'Pálpataka']","['Pálpatak', 'Pálpataka']",Pálpataka,46.5109,25.2127,"Pálpataka, 0"
2,Már a háromszéki prefektusnak is nagyon elege ...,1,2019-09-12,2019. szeptember 12.,https://szekelyhon.ro/aktualis/mar-a-haromszek...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Háromszék', 'medve']",Már a háromszéki prefektusnak is nagyon elege ...,...,1,0,0,0,Kovászna,Kovászna,Kovászna,46.7267,25.7192,"Kovászna, 1"
3,Kútba esett egy medve BibarcfalvánBeleesett eg...,2,2019-09-13,2019. szeptember 13.,https://szekelyhon.ro/aktualis/kutba-esett-egy...,https://media.szekelyhon.ro/pictures/haromszek...,aktualis,haromszek,"['Háromszék', 'medve']",Kútba esett egy medve Bibarcfalván,...,1,0,0,0,Bibarcfalva,Bibarcfalva,Bibarcfalva,46.0933,25.6525,"Bibarcfalva, 2"
4,Székelyudvarhely utcáin portyázott a bocsos me...,3,2019-09-16,2019. szeptember 16.,https://szekelyhon.ro/aktualis/szekelyudvarhel...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Székelyudvarhely']",Székelyudvarhely utcáin portyázott a bocsos medve,...,1,0,0,0,"['Szabéd', 'Székelyudvarhely']","['Szabéd', 'Székelyudvarhely']",Szabéd,46.6641,24.4442,"Szabéd, 3"
5,Székelyudvarhely utcáin portyázott a bocsos me...,3,2019-09-16,2019. szeptember 16.,https://szekelyhon.ro/aktualis/szekelyudvarhel...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Székelyudvarhely']",Székelyudvarhely utcáin portyázott a bocsos medve,...,1,0,0,0,"['Szabéd', 'Székelyudvarhely']","['Szabéd', 'Székelyudvarhely']",Székelyudvarhely,46.305,25.2926,"Székelyudvarhely, 3"
6,Letörte a csűr ajtaját is a medve a juhokértCs...,4,2019-09-17,2019. szeptember 17.,https://szekelyhon.ro/aktualis/letorte-a-csur-...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Homoródalmás', 'medve']",Letörte a csűr ajtaját is a medve a juhokért,...,1,3,0,0,Homoródalmás,Homoródalmás,Homoródalmás,46.2344,25.457,"Homoródalmás, 4"
7,Medve támadt egy lengyel turistáraLengyel turi...,5,2019-09-17,2019. szeptember 17.,https://szekelyhon.ro/vilag/medve-tamadt-egy-l...,https://media.szekelyhon.ro/pictures/udvarhely...,vilag,udvarhely,"['Belföld', 'medve']",Medve támadt egy lengyel turistára,...,1,3,0,0,NaN,NaN,,,,"N/A, 5"
8,Amikor négy tehén és három medve néz farkassze...,6,2019-09-18,2019. szeptember 18.,https://szekelyhon.ro/aktualis/amikor-negy-teh...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Háromszék', 'medve']",Amikor négy tehén és három medve néz farkassze...,...,1,0,0,0,NaN,NaN,,,,"N/A, 6"
9,Hárombocsos anyamedve közelítette meg a csíkso...,7,2019-09-18,2019. szeptember 18.,https://szekelyhon.ro/aktualis/harombocsos-any...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Csíksomlyó', 'medve']",Hárombocsos anyamedve közelítette meg a csíkso...,...,1,0,0,0,Csíksomlyó,Csíksomlyó,Csíksomlyó,46.3753,25.8193,"Csíksomlyó, 7"


Save data for loading in viz app

In [15]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [16]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [17]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')